In [3]:
import tensorflow
import cv2
import numpy as np

img_path = "test_img.jpg"
img = cv2.imread(img_path)
orig = img.copy()
(H, W) = img.shape[:2]

newW, newH = (320, 320)
rW = W/newW
rH = H/newH
resized = cv2.resize(img, (newW, newH))

net = cv2.dnn.readNet("frozen_east_text_detection.pb")
blob = cv2.dnn.blobFromImage(resized, 1.0, (newW, newH), (123.68, 116.78, 103.94), swapRB = True, crop = False)
net.setInput(blob)
layers = ["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"]
(scores, geometry) = net.forward(layers)

rects = []
confidences = []


In [6]:
for y in range(scores.shape[2]):
    for x in range(scores.shape[3]):
        if scores[0, 0, y, x] < 0.5:
            continue

        angle = geometry[0,4, y, x]
        cos = np.cos(angle)
        sin = np.sin(angle)

        h = geometry[0,0, y, x] + geometry[0, 2, y, x]
        w = geometry[0,1,y,x] + geometry[0, 3, y,x]

        offsetX = x * 4.0
        offsetY = y*4.0

        endX = int(offsetX + cos*geometry[0, 1, y, x] + sin*geometry[0,2,y,x])
        endY = int(offsetY - sin*geometry[0, 1, y,x] + cos* geometry[0,2,y,x])    
        startX = int(endX - w)
        startY = int(endY - h)

        startX = int(startX * rW)
        endX = int(endX * rW)
        startY = int(startY * rH)
        endY = int(endY * rH)

        rects.append([startX, startY, endX, endY])
        confidences.append(float(scores[0, 0, y, x]))

In [12]:
import pytesseract

In [11]:
boxes = cv2.dnn.NMSBoxes(rects, confidences, 0.5, 0.4)

if len(boxes) == 0:
    boxes = []
else :
    boxes = boxes.flatten().tolist()

config = "--oem 1 --psm 7"

for idx in boxes :
    x1, y1, x2, y2 = rects[idx]
    roi = orig[y1:y2, x1:x2]

    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    text = pytesseract.image_to_string(roi_gray, config=config)

    cv2.rectangle(orig, (x1, y1), (x2, y2), (0, 255, 0), 2)

import matplotlib.pyplot as plt

rgb = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
plt.imshow(rgb)
plt.show()

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.